In [ ]:
pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install deepface

In [ ]:
import tenseal as ts # pip install tenseal
from deepface import DeepFace #!pip install deepface
import base64

# Requirements

# Finding embeddings

We are going to find vector representations of facial images. This will be done in the client side.

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
img1_embedding = DeepFace.represent("img1.jpg", model_name = 'Facenet')
img2_embedding = DeepFace.represent("img2.jpg", model_name = 'Facenet')
print(img1_embedding)
print(img2_embedding)

[{'embedding': [0.859748125076294, 0.9605176448822021, 1.3458293676376343, -1.5352717638015747, 0.4913597106933594, -0.48713383078575134, 0.8994464874267578, 0.780159056186676, 1.8225899934768677, 0.9173871874809265, -0.9213154911994934, 0.41336697340011597, -1.013060450553894, -0.6539306640625, -1.8478519916534424, 0.431438148021698, -1.550025224685669, -0.35439908504486084, 1.0887333154678345, -0.35816821455955505, 1.7201800346374512, -0.2550181746482849, -0.7676998376846313, -1.3535757064819336, -2.4413671493530273, 2.2766613960266113, -0.4840104281902313, -0.8013039231300354, -0.7300171852111816, 0.6920195817947388, -0.7333544492721558, 1.3712615966796875, 0.8122579455375671, -1.8422991037368774, 0.2873917520046234, -0.07752888649702072, -1.4163769483566284, 0.437186062335968, 1.4368438720703125, 0.09004097431898117, -0.6153519749641418, 0.4684123396873474, -1.3318177461624146, -0.4740756154060364, 0.05478878319263458, 1.7802492380142212, -2.0101234912872314, 1.3570033311843872, -0

# Commons

# Initialization

We are going to generate secret - public key pair in this stage. This will be done in the client side.

In [ ]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [ ]:
secret_context = context.serialize(save_secret_key = True)

In [ ]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()

# Encryption

We are going to apply homomorphic encryption to facial embeddings. This will be done in the client side.

Then, homomorphic encrypted facial embeddings will be stored in the cloud.

In [ ]:
context = ts.context_from(secret_context)
print(context)
print(img1_embedding)

[{'embedding': [0.859748125076294, 0.9605176448822021, 1.3458293676376343, -1.5352717638015747, 0.4913597106933594, -0.48713383078575134, 0.8994464874267578, 0.780159056186676, 1.8225899934768677, 0.9173871874809265, -0.9213154911994934, 0.41336697340011597, -1.013060450553894, -0.6539306640625, -1.8478519916534424, 0.431438148021698, -1.550025224685669, -0.35439908504486084, 1.0887333154678345, -0.35816821455955505, 1.7201800346374512, -0.2550181746482849, -0.7676998376846313, -1.3535757064819336, -2.4413671493530273, 2.2766613960266113, -0.4840104281902313, -0.8013039231300354, -0.7300171852111816, 0.6920195817947388, -0.7333544492721558, 1.3712615966796875, 0.8122579455375671, -1.8422991037368774, 0.2873917520046234, -0.07752888649702072, -1.4163769483566284, 0.437186062335968, 1.4368438720703125, 0.09004097431898117, -0.6153519749641418, 0.4684123396873474, -1.3318177461624146, -0.4740756154060364, 0.05478878319263458, 1.7802492380142212, -2.0101234912872314, 1.3570033311843872, -0

In [ ]:
enc_v1 = ts.ckks_vector(context, img1_embedding[0]['embedding'])
enc_v2 = ts.ckks_vector(context, img2_embedding[0]['embedding'])
print(enc_v1)
print(enc_v2)

# Calculations

Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Notice that we just have public key here and we don't have secret key.

In [ ]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [ ]:
#we must not decrypt the homomorphic encrypted euclidean squared value in this stage
#because we don't have the secret key. check this operation. it should throw an exception!

try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))

# Decryption

Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.

In [ ]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [ ]:
euclidean_squared_plain

63.97125405785755

In [ ]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are same person
